In [1]:
import torch
import syft as sy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch import optim
from tqdm import tqdm
device = torch.device("cuda")
torch.manual_seed(5)

In [2]:
torch.cuda.is_available()

True

In [3]:
dataset =  pd.read_csv('shuttle-unsupervised-ad.csv', header=None)
dataset.columns = ["a",'b','c','d', 'e', 'f' , 'g', 'h', 'i', 'j']
dataset.head()

,a,b,c,d,e,f,g,h,i,j
0,108.0,3.0,109.0,0.0,72.0,7.0,1.0,36.0,36.0,o
1,81.0,0.0,84.0,0.0,-14.0,-2.0,4.0,100.0,96.0,o
2,81.0,0.0,84.0,0.0,-20.0,16.0,4.0,105.0,102.0,o
3,76.0,-1.0,81.0,0.0,-42.0,-3.0,5.0,125.0,120.0,o
4,105.0,0.0,107.0,2.0,70.0,0.0,1.0,37.0,36.0,o


In [4]:
dataset.replace({"o":1, "n":0}, inplace=True)

In [5]:
dataset = (dataset - dataset.min())/(dataset.max() - dataset.min())

In [6]:
dataset.head()

,a,b,c,d,e,f,g,h,i,j
0,0.843750,0.695558,0.687500,0.507015,0.416667,0.458156,0.206107,0.624398,0.630225,1.0
1,0.562500,0.695092,0.492188,0.507015,0.278846,0.457834,0.229008,0.727127,0.726688,1.0
2,0.562500,0.695092,0.492188,0.507015,0.269231,0.458478,0.229008,0.735152,0.736334,1.0
3,0.510417,0.694936,0.468750,0.507015,0.233974,0.457799,0.236641,0.767255,0.765273,1.0
4,0.812500,0.695092,0.671875,0.507272,0.413462,0.457906,0.206107,0.626003,0.630225,1.0


In [7]:
data_client1 = dataset.sample(frac=0.5)
data_client2 = dataset[~dataset.index.isin(data_client1.index)]

In [8]:
features_client1, target_client1 = data_client1[[cols for cols in data_client1.columns if cols!='j']], data_client1.j
features_client2, target_client2 = data_client2[[cols for cols in data_client2.columns if cols!='j']], data_client2.j

In [9]:
x_client1 = Variable(torch.from_numpy(features_client1.values), requires_grad = False).cuda().float()
x_client2 = Variable(torch.from_numpy(features_client2.values), requires_grad = False).cuda().float()

In [10]:
def model(In_size, H_size=14):
    if type(H_size) is not list:
        model = torch.nn.Sequential(
                    torch.nn.Linear(In_size, H_size),
                    torch.nn.ReLU(),
                    torch.nn.Linear(H_size, H_size),
                    torch.nn.Tanh(),
                    torch.nn.Linear(H_size, H_size),
                    torch.nn.ReLU(),
                    torch.nn.Linear(H_size, In_size))
            
    
    return model
    
loss_fn = torch.nn.MSELoss(size_average = False)

C:\Users\Shubham\Anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [11]:
mod = model(dataset.shape[1] - 1, 14)
opt = optim.SGD(params=mod.parameters(), lr=1e-5)

In [12]:
mod.cuda()

Sequential(
  (0): Linear(in_features=9, out_features=14, bias=True)
  (1): ReLU()
  (2): Linear(in_features=14, out_features=14, bias=True)
  (3): Tanh()
  (4): Linear(in_features=14, out_features=14, bias=True)
  (5): ReLU()
  (6): Linear(in_features=14, out_features=9, bias=True)
)

In [13]:
print(mod)

Sequential(
  (0): Linear(in_features=9, out_features=14, bias=True)
  (1): ReLU()
  (2): Linear(in_features=14, out_features=14, bias=True)
  (3): Tanh()
  (4): Linear(in_features=14, out_features=14, bias=True)
  (5): ReLU()
  (6): Linear(in_features=14, out_features=9, bias=True)
)


In [14]:
hook = sy.TorchHook(torch)

client_1 = sy.VirtualWorker(hook, id="client_1")
client_2 = sy.VirtualWorker(hook, id="client_2")

In [15]:
epoch = 20
batch_size= 64
batches = [[(x_client1[i:( i + batch_size)].send(client_1),), (x_client2[i:( i + batch_size)].send(client_2),)] for i in range(0, len(x_client2), batch_size)]

In [16]:
for __ in range(epoch):
    print('Epoch start\n')
    
    for batch in tqdm(batches):
        
        for data in batch:
            data = data[0]
            mod.send(data.location)
            opt.zero_grad()
            
            pred = mod.forward(data)
            loss = loss_fn(pred, data)

            mod.zero_grad()
            loss.backward()
            
            opt.step()
            
            mod.get()
            print(loss.get())

Epoch start



  0%|                                                                                          | 0/363 [00:00<?, ?it/s]

tensor(223.2797, device='cuda:0', requires_grad=True)
tensor(251.9968, device='cuda:0', requires_grad=True)


  0%|▏                                                                                 | 1/363 [00:00<02:54,  2.07it/s]

tensor(220.9276, device='cuda:0', requires_grad=True)
tensor(250.9353, device='cuda:0', requires_grad=True)
tensor(219.4187, device='cuda:0', requires_grad=True)
tensor(249.8195, device='cuda:0', requires_grad=True)


...

 99%|███████████████████████████████████████████████████████████████████████████████▌| 361/363 [00:38<00:00,  9.80it/s]

tensor(1.1461, device='cuda:0', requires_grad=True)
tensor(0.9913, device='cuda:0', requires_grad=True)


100%|███████████████████████████████████████████████████████████████████████████████▊| 362/363 [00:38<00:00,  9.75it/s]

tensor(1.1791, device='cuda:0', requires_grad=True)
tensor(1.0865, device='cuda:0', requires_grad=True)


100%|████████████████████████████████████████████████████████████████████████████████| 363/363 [00:38<00:00,  9.48it/s]


In [17]:
res = mod.forward(x.cuda()[:2])

NameError: name 'x' is not defined

In [ ]:
loss_fn = loss(res, x)

In [ ]:
loss_fn.backward()

In [ ]:
res = mod.forward(x)

In [ ]:
loss_fn = loss(res, x)

In [ ]:
loss_fn

In [ ]:
(res[0] - x.cuda()[0])